## Comparing the MSE of Bayes vs Frequentist regression

In [1]:
import pandas as pd
import numpy as np
import pymc as pm

In [2]:
train = pd.read_csv('wr_train.csv')

In [4]:
train

,Overall,yds_game,Rec.TD,RAS,P_five
0,24,1.092823,2.094429,0.895544,1
1,39,-0.005854,-1.536779,0.475042,1
2,60,1.551306,1.186627,0.095565,1
3,78,-0.703085,-0.402027,-1.242861,1
4,82,0.633253,-0.628977,1.064771,0
...,...,...,...,...,...
107,128,0.650908,0.505776,-0.232631,0
108,142,0.819036,0.051874,-0.601852,0
109,151,-1.517926,-0.628977,1.049386,1
110,168,-0.876996,-0.402027,0.367353,0


In [3]:
test = pd.read_csv('wr_test.csv')

In [24]:
meany = train['Overall'].mean()
scaley = train['Overall'].std()
zy = ((train['Overall']-meany)/scaley).values

In [26]:
meany = test['Overall'].mean()
scaley = test['Overall'].std()
zytest = ((test['Overall']-meany)/scaley).values

In [34]:
with pm.Model() as wr_model:
    Y = pm.MutableData(name = 'Y', value = zy)
    X1 = pm.MutableData(name = 'X1', value = train['yds_game'])
    X2 = pm.MutableData(name = 'X2', value = train['Rec.TD'])
    X3 = pm.MutableData(name = 'X3', value = train['RAS'])
    X4 = pm.MutableData(name = 'X4', value = train['P_five'])
    
    beta0 = pm.Normal('beta0', mu=0, sigma=2) 
    beta1 = pm.Normal('beta1', mu=0, sigma=2) 
    beta2 = pm.Normal('beta2', mu=0, sigma=2)
    beta3 = pm.Normal('beta3', mu=0, sigma=2)
    beta4 = pm.Normal('beta4', mu=0, sigma=2)

    mu = beta0 + beta1*X1 + beta2*X2 + beta3*X3 + beta4*X4
    
    nu = pm.Exponential('nu', 1/29.)
    sigma = pm.Uniform('sigma', 10**-5, 10)
    
    likelihood = pm.StudentT('likelihood', nu=nu, mu=mu, lam=1/sigma**2, observed = Y)
    trace = pm.sample(1000, cores = 4,target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1, beta2, beta3, beta4, nu, sigma]


/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/connornickol/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 83 seconds.


In [35]:
with wr_model:
    trace.extend(pm.sample_prior_predictive())
    trace.extend(pm.sample_posterior_predictive(trace))

Sampling: [beta0, beta1, beta2, beta3, beta4, likelihood, nu, sigma]
Sampling: [likelihood]


In [37]:
pm.set_data(new_data={'X1':test['yds_game'],'X2':test['Rec.TD'],'X3':test['RAS'],'X4':test['P_five'], 'Y':zytest}, model=wr_model)
ppc_power = pm.sample_posterior_predictive(trace, model=wr_model, var_names=['likelihood'])

Sampling: [likelihood]


In [38]:
predict_p = pd.DataFrame({
    'predict':ppc_power['posterior_predictive']['likelihood'].mean(axis=1)[0].to_numpy()})

In [39]:
test['predict'] = predict_p

In [40]:
## MSE
((test['Overall'] - test['predict'])**2).mean()

8424.527154980073

In [41]:
test

,Overall,yds_game,Rec.TD,RAS,P_five,predict
0,20,0.059309,0.051874,0.869904,1,-0.286131
1,27,0.242129,2.775281,0.377609,1,-1.001572
2,34,2.587648,-0.402027,0.710934,1,-0.692588
3,49,0.369245,0.505776,1.039130,1,-0.480077
4,57,0.427913,0.505776,-0.540315,1,-0.292397
5,77,-1.680242,-1.309829,0.146846,1,0.426419
6,82,0.502286,-0.402027,0.551964,1,-0.289729
7,85,-0.129438,-0.628977,-1.001842,1,0.104289
8,89,-1.156952,0.959677,1.162204,1,-0.329882
9,91,-1.036192,-1.536779,-0.212119,1,0.458558
